# Experiment 4: Latest features on all statistical classifiers

In [3]:
reviews_set, fake_reviews, genuine_reviews, unused_genuine_reviews = get_balanced_dataset()
print("fake:", len(fake_reviews))
print("real:", len(genuine_reviews))
print("all:", len(concatted_reviews))
print("unused real:", len(unused_genuine_reviews))

fake: 80466
real: 80467
all: 160933
unused real: 447665


In [1]:
targets = [x.label for x in reviews_set]

NameError: name 'concatted_reviews' is not defined

## Linear Discriminant Analysis

In [2]:
from latest_feature_extraction import get_features_maker

In [58]:
get_features = get_features_maker(reviews_set, 775)
predictor_features = get_features(reviews_set)

Here we can only make our bag of words features so big before we crash with MemoryError

In [60]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
import numpy
linearDA = LinearDiscriminantAnalysis()
results = cross_validate(linearDA, predictor_features.toarray(), numpy.asarray(targets), cv=2,
                         return_train_score=False)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [61]:
sum(x for x in results['test_score'])/2

0.6899020151921419

## Feature scaling

We normalise all our features to be between one and zero. We need to do this to suppress the mega features vs tiny features situation. Most classifiers use Euclidian distance, which has no knowledge of the units being used. 

Our sparse features are problematic here because we need to convert our data to array for standardisation. Since we cannot do this with sparse data without MemoryErrors, we will use dense features for our bag of words from experiment 2.

In [75]:
from latest_feature_extraction import dense_features_maker
from exp2_feature_extraction import find_words

corpus_words = [find_words(x.review_content) for x in concatted_reviews]

In [76]:
dense_features_getter = dense_features_maker(corpus_words, 75)

In [77]:
predictor_features_dense = [dense_features_getter(x) for x in corpus_words]

In [78]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(predictor_features_dense)

Warning, because we're going straight for cross validation, the test set is part of what fits the scaler

In [79]:
scaled_features = scaler.transform(predictor_features_dense)

## Support Vector Machine

In [80]:
from sklearn.svm import LinearSVC
svc = LinearSVC(max_iter=10000)

In [81]:
# Should already be imported
from sklearn.model_selection import cross_validate

In [82]:
targets = [x.label for x in concatted_reviews]

In [83]:
cross_validate(svc, scaled_features, targets, cv=2, return_train_score=False)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'fit_time': array([480.73824811, 469.01575494]),
 'score_time': array([0.02557325, 0.02270246]),
 'test_score': array([0.62932631, 0.62630179])}

When attempting to use LDA topics to have more dense features for our SVM, we have a very low accuracy. For some reason it is ?% better to just use bag of words again.

## Logistic regression

In [84]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()

In [86]:
cross_validate(logistic_regression, scaled_features, targets, cv=2, return_train_score=False)

/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/stefan/anaconda3/envs/lucas/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'fit_time': array([0.88004017, 0.90394258]),
 'score_time': array([0.02301192, 0.02334046]),
 'test_score': array([0.63025837, 0.62705988])}

## K Nearest Neighbors

In [73]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()

In [87]:
cross_validate(knn_classifier, scaled_features, targets, cv=2, return_train_score=False)

{'fit_time': array([26.47510648, 23.95157218]),
 'score_time': array([1133.47025609, 1098.35629988]),
 'test_score': array([0.58381697, 0.58850943])}

Note convergencewarning